# Assignment 5: Anomaly Detection with Negative Selection

### Assignment description:

In this assignment, you will use the same Textor algorithm from the previous assignment to detect anomalies in a different data set. We will use a fetal monitoring dataset (known as a cardiotocography) from the UC Irvine Machine Learning Repository. 

### Assignment goals:

1. Apply the Textor algorithm to a practical example
2. Use ROC analysis to score the detector

### Assignment question overview:

1. Write a function to process the cardiotocography data set. [Question 1](#question1)
2. Paste any previous functions or write additional helper functions (not-graded). [Question 2](#question2)
3. Write a function to calculate AUC values over an interval of $r$ values. [Question 3](#question3)


## Cardiotocography Dataset

A cardiotocography is a technical way to measure the fetal heart rate (FHR) and the uterine contractions (UC) during pregnancy. Obstetricians then classify these readings as either normal, suspect, and pathologic. Figure 1 shows the display of a cardiotocograph (CTG), the FHR is shown in orange and the UC is shown in green. Figure 2 shows the output of a typical CTG where the line labeled **A** is the FHR and the line labeled **D** is the UC. More details on the data set are here: http://odds.cs.stonybrook.edu/cardiotocogrpahy-dataset/. 

The data consists of 21 real-valued variables, outliers and inliers. Each row is a sample, each column is an observed variable , and the training file includes the ground truth labels of the samples. The data are in: ``cardio_train.csv`` and ``cardio_test.csv``. 

<table><tr>
<td> 
  <p align="center" style="padding: 10px">
    <img alt="Forwarding" src="images/cardio.jpg" width="250">
    <br>
    <em style="color: grey">Figure 1: Cardiotocograph display</em>
  </p> 
</td>
<td> 
  <p align="center">
    <img alt="Routing" src="images/CTG_Output.jpg" width="650">
    <br>
    <em style="color: grey">Figure 2: Cardiotocograph output</em>
  </p> 
</td>
</tr></table>

*Image sources: Wikipedia*



## Data Processing

In order for the continuous data to work well with the negative selection algorithm, we will need to bin the values into categorical variables. For consistency across assignments, we will simply bin each of the variables in to 10 bins. This will produce strings of length 21 symbols and an alphabet size of 10 characters (e.g. A indicates the lowest category and J indicates the highest). 

Next, we will need to format the data in such a way that the new train and test data may be read directly by Textor's algorithm used in the previous assignment. Recall that both the train and the test data for the languages were ``.csv`` files with one string of length 10 per line. Here, we would like there to be one string of length 21 per line. These strings will correspond to the 21 variables in a row of either ``cardio_train.csv`` or ``cardio_test.csv``. 

### Creating Bins with pandas

To avoid ambiguity in how the data can be binned, we suggest that you use the ``cut`` function from pandas. Note that this function allows for a labels parameter to be passed. To map our binned data to characters we can define the labels array as follows:

``labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']``.

After defining labels we can call ``pandas.cut(x, bins=10, labels)``, where ``x`` is the data column.

<a id='question1'></a>
## Question 1:

Write the function ``process_data(file)`` which processes either ``cardio_train.csv`` or ``cardio_test.csv`` into the format specified in the "Data Processing" section. This function returns the string ``processed_file`` which is a path to the final processed file. This function will not be directly graded, but will be used in the grading of later questions. The processed file should be in ``.csv`` format with one string of length 21 per row. The final processed file should look similar to the strings below. 

**Do not include headers or indices in your final processed file. The file should contain only strings.**

```
EEDAECEAACACIBJDBEDDA
EEBAFCEAACACIBJCBEDDA
EEBAFBEAADAEHAFGAEDDA
EEDAFAEAADADHAFFAEDDA
EECBFBEACBACDDDCAEDDA
EEEEEAEACBABFBEBAEDEA
DDDICDECBCABGBGDAFCDE
DDCHCCEEBCABGBGBAECDC
DDDJDCEABCACIAHCAFCDD
DDBJDCEFBCABGAEDADCCC
DDCJDCEDCBABHBGDBDCDB
DDCICCEFCBABJAJFADCCB
EEACAHEACIAAJAJFADAAF
...
```

In [24]:
import pandas as pd
import csv

def process_data(file):
    labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    df = pd.read_csv(file, skiprows=1, header = None, index_col = False)
#     print(df)
    df.drop(df.columns[0], axis=1, inplace=True)
    if file == 'cardio_test.csv':
        df.drop(df.columns[-1], axis=1, inplace=True)
#     print(df)
    for col in df.columns:
        df[col] = pd.cut(df[col], bins=10, labels=labels)
    
    
    string_output = ''
    for index, row in df.iterrows():
        string_output += ''.join(row) + '\n'
#     print(string_output)
    lines = string_output.split('\n')

    with open('formatted_' + file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',')

        for line in lines:
            csvwriter.writerow(line.split())
    
    '''
    Function to process the data in a given file. 
    Inputs:
        file: path to either test or train file
    Outputs:
        processed_file: path to processed version of the file
    '''
    
    # your code here
    
    
    return 'formatted_' + file

'formatted_cardio_test.csv'

## Implementing Negative Selection Recap

Recall from the previous assignment that we used a python subprocess to use Textor's ``negsel.jar`` file. To start a subprocess, we will first need to import the ``subprocess`` module and the ``run`` and ``PIPE`` submodules. Then, we will need to run the process with an array of arguments and the opened input file. We will use the Textor example ([here](http://johannes-textor.name/negativeselection.html)) to demonstrate how subprocesses work.

The following command trains the negative selection algorithm on the ``english.train`` data set and tests on the ``hiligaynon.test`` data set from the previous assignment.

``java -jar negsel.jar -c -n 10 -r 4 -self english.train < hiligaynon.test ``

In this assignment, you will used the processed train and test data to run this same command. Recall that to run this subprocess, we will need to first build an argument list from the command. This allows the subprocess command to parse the command and flags. Using the command given above, we will construct the following argument list. 

``args = ['java', '-jar', 'negsel.jar', '-c', '-n', '10', '-r','4', '-self', 'negsel_src/tests/english.train']``

Next, we will need to open the test file to read from. This step is necessary as the argument list will not recognize the redirection from stdin (``< hiligaynon.test``).

``input_file = open('negsel_src/tests/hiligaynon.test', 'r')``

Finally, we can use the ``run`` submodule to call the command and pipe the output. 

``result = run(args, stdout=PIPE, stdin=input_file, universal_newlines=True)``

The final results will be stored in ``result.stdout``. This can be saved and parsed later as needed. 

Refer to the previous assignment for more information on how to run the code as well as a full running example.

**REMEMBER TO ADJUST THE STRING LENGTH FOR THIS ASSIGNMENT.**

## Exploring Model Parameterization

In the previous assignment, we used $r=4$ contiguous bits for all of our testing. However, this does not ensure optimal anomaly detection. In this assignment, we will investigate the tuning of the $r$ parameter for the *$r$-contiguous* patterns. 
 

### $r$-contiguous Patterns

Textor in _"A Comparative Study of Negative Selection Based Anomaly Detection in Sequence Data"_ defines the $r$-contiguous pattern as follows.

<blockquote>An $r$-contiguous pattern is a string $d\in \Sigma^l$. It matches another string $s\in  \Sigma^l$ if $d$ and $s$ are identical in at least $r$ contiguous positions, i.e., if there is an $i\in \{1,...,l-r+1\}$ such that the substrings of length $r$ of $s$ and $d$ starting at the $i$-th position are equal. </blockquote> 

### Scoring the Detector 

Since our strings are of length 21, we will consider $r\in [2,10]$. For simplicity, we will use the *AUC* as described in the previous assignment. Recall that the ROC curve is created by plotting the false positive (FP) rate and the true positive (TP) rate over an interval of values of $\theta$. Generally, a meaningful classifier has an area under the curve (AUC) value greater than 0.5, and a AUC value close to 1 signifies a near-perfect classifier. 

You may use the ``fp_tp_calc(file, theta)`` and ``auc_calc(file)`` functions from the previous assignment to calculate the AUC values. Additionally, we will let $\theta \in [0,10]$, and $ \theta \in \mathbb{Z}$. 

<a id='question2'></a>
## Question 2: Using previous functions

Use the answer box below to paste in any functions that you would like to use for Question 2. You may also use this area to write any additional helper functions and/or edit your old functions to generalize to this problem and dataset (i.e., by adding a new parameter to handle the fact that r is an additional input variable). Labels for this dataset may be found in the file cardio.labels.



In [55]:
## PASTE ANY FUNCTIONS YOU WOULD LIKE TO USE HERE
import numpy as np
import subprocess
from subprocess import run, PIPE
from sklearn import metrics

def fp_tp_calc(file, theta, r):
    '''
    Function to calculate the false positive and true positive rate for a given test set and value of theta.
    Inputs: 
        file: path to the test file
        theta: threshold value for ROC analysis
    Output:
        fp_tp : python tuple of the FPR and TPR given by (fp, tp)
    '''
    
    args = ['java', '-jar', 'negsel.jar','-n','21', '-r', str(r), '-self', 'formatted_cardio_train.csv']
    with open(file, 'r') as input_file:
        result = run(args, stdout=PIPE, stderr=PIPE, stdin=input_file, universal_newlines=True)
    anomaly_scores = result.stdout.splitlines()
#     print(anomaly_scores)
    anomaly_scores = [float(score) for score in anomaly_scores]
    with open('cardio.labels', 'r') as f:
        labels = np.array([int(label) for label in f.read().splitlines()])
    normal_scores = [score for score, label in zip(anomaly_scores, labels) if label == 0]
    anomalous_scores = [score for score, label in zip(anomaly_scores, labels) if label == 1]
    
    false_positives = sum(score>theta for score in normal_scores)
    true_positives = sum(score>theta for score in anomalous_scores)
    fpr = false_positives / labels.tolist().count(0)
    tpr = true_positives / labels.tolist().count(1)
    fp_tp = (fpr, tpr)
    # your code here
    
    return fp_tp

def auc_calc(file, r):
    '''
    Function to calculate the AUC for a given test set over the theta interval of [0,10].
    Inputs: 
        file: path to the test file
    Output:
        auc : area under the curve for the ROC curve of the test set
    '''
    fprs = []
    tprs = []
    for theta in range(11):
#         print(theta)
        fp,tp = fp_tp_calc(file, theta, r)
        fprs.append(fp)
        tprs.append(tp)
    auc = metrics.auc(fprs, tprs)
        
    
    
    # your code here
    

    return auc
# your code here


<a id='question3'></a>
## Question 3:

Write the function ``r_auc_tuple(file)`` that calculates a list of tuples for each value of $r\in [2,10]$ over $\theta \in [0,10]$ for a given test file. This will result in a list of 10 tuples where the first element in each tuple is the value of $r$ and the second element is the AUC value.  This list will be returned as ``tuples_list`` and will be graded based on accuracy. The labels for the test file are in ``cardio.labels``.

For reference, the following command should return the AUC value for $r=2$: ``r_auc_tuple('cardio_test.csv')[0][1]``.

**Remember to use the ``process_data(file)`` function to run the Textor algorithm with the correct files.**

In [56]:
def r_auc_tuple(file):
    use_file = process_data(file)
    process_data('cardio_train.csv')

    
    tuples_list = []
    
    for r in range(2,11):
        tuples_list.append((r, auc_calc(use_file, r)))

        
    '''
    Function to compute the auc values for various values of r.
    Inputs: 
        file: path to the test file
    Outputs:
        tuples_list: a list of tuples where the first value in each tuple is r and the second value is the auc
    '''
    # your code here
    
    
    return tuples_list
# tuples = r_auc_tuple('cardio_test.csv')
# print(tuples)
# print(type(tuples))
# assert len(tuples) == 9
# assert round(tuples[0][1],2) == 0.75
# assert round(tuples[7][1],2) == 0.0
# print("All test cases passed")

In [57]:
'''
Check the output of your functions.
'''
tuples = r_auc_tuple('cardio_test.csv')
print(tuples)
assert len(tuples) == 9
assert round(tuples[0][1],2) == 0.75
assert round(tuples[7][1],2) == 0.0
print("All test cases passed")

[(2, 0.7548143650242887), (3, 0.7168459403192228), (4, 0.2917071478140181), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.0)]
All test cases passed


In [23]:
'''
This cell contains hidden tests, which are run on submission. 
Hidden test cases check the tuples for the correct AUC values using the cardio_test.tsv file.
'''


'\nThis cell contains hidden tests, which are run on submission. \nHidden test cases check the tuples for the correct AUC values using the cardio_test.tsv file.\n'

In [ ]:
'''
This cell contains hidden tests, which are run on submission. 
Hidden test cases check the tuples for the correct AUC values using the cardio_test.tsv file.
'''


In [ ]:
'''
This cell contains hidden tests, which are run on submission. 
Hidden test cases check the tuples for the correct AUC values using the cardio_test.tsv file.
'''
